In [1]:
#Importando Bibliotecas
import requests
from psycopg2.extras import execute_values
import psycopg2
import pandas as pd

pd.set_option('display.max_columns', None)

In [2]:
# Recebe Links e tranforma dados em texto
link_versao = requests.get('https://ddragon.leagueoflegends.com/api/versions.json')
link_versao_json = link_versao.json()
versao_validada = []

#Valido Quais versões estão disponíveis e coloco as disponíveis em uma lista
for i in link_versao_json:
    campeao_string_link = "https://ddragon.leagueoflegends.com/cdn/" + i + "/data/pt_BR/champion.json"
    if requests.get(campeao_string_link).status_code == 200:
        versao_validada.append(i)

In [3]:
#Define parametros do Postgre para inserção dos dados
db_params  = psycopg2.connect(database = "APILol",
                 host = "localhost", 
                 user = "postgres",
                 password = "d@d0s!2025",
                 port = "5432")

In [4]:
# Criando uma lista e colocando as informações escolhidas de cada campeão de cada versãro válidada em um dataframe
campeoes = []

for j in versao_validada:
    
    campeao_string_link = requests.get("https://ddragon.leagueoflegends.com/cdn/" + j + "/data/pt_BR/champion.json")
    lista_geral_campeoes = campeao_string_link.json()
    lista_geral_campeoes_data = lista_geral_campeoes['data']
    url_imagem = "https://ddragon.leagueoflegends.com/cdn/" + j + "/img/champion/"
        
    for champion, data in lista_geral_campeoes_data.items():
        id_unico = j.replace(".", "0") + data['key'].zfill(5)
        campeoes.append({
            'versao_api': j,
            'id_unico': id_unico,
            'id': data['id'],
            'chave': data['key'],
            'nome': data['name'],
            'titulo': data['title'],
            'tipo': data.get('partype', None),
            'ataque': data['info']['attack'],
            'defesa': data['info']['defense'],
            'magia': data['info']['magic'],
            'dificuldade': data['info']['difficulty'],
            'hp': data['stats']['hp'],
            'hp_por_nivel': data['stats']['hpperlevel'],
            'mp': data['stats']['mp'],
            'mp_por_nivel': data['stats']['mpperlevel'],
            'velocidade_movimento': data['stats']['movespeed'],
            'armadura': data['stats']['armor'],
            'armadura_por_nivel': data['stats']['armorperlevel'],
            'mr': data['stats']['spellblock'],
            'mr_por_nivel': data['stats']['spellblockperlevel'],
            'ataque_range': data['stats']['attackrange'],
            'regeneracao_hp': data['stats']['hpregen'],
            'regeneracao_hp_nivel': data['stats']['hpregenperlevel'],
            'regeneracao_mp': data['stats']['mpregen'],
            'renegeracao_mp_nivel': data['stats']['mpregenperlevel'],
            'critico': data['stats']['crit'],
            'critico_por_nivel': data['stats']['critperlevel'],
            'dano_ataque': data['stats']['attackdamage'],
            'dano_ataque_por_nivel': data['stats']['attackdamageperlevel'],
            'velocidade_ataque': data.get('stats', {}).get('attackspeed', None),
            'velocidade_ataque_por_nivel': data['stats']['attackspeedperlevel'],   
            'imagem': url_imagem + data['image']['full']
        })



In [5]:
#Criando e Tratando DataFrame

df_campeao = pd.DataFrame(campeoes) 
df_campeao['titulo'] = df_campeao['titulo'].str.title()

In [6]:
#Abrindo conexão no banco e criando tabela caso não exista

cursor = db_params.cursor()

# Comando SQL para criar uma tabela
create_table_query = '''
CREATE TABLE IF NOT EXISTS tabelacampeao (
    ID_UNICO bigint PRIMARY KEY,
    VERSAO_API VARCHAR(100),
    ID VARCHAR(100),
    CODIGO_CAMPEAO VARCHAR(100),
    NOME_CAMPEAO VARCHAR(100),
    TITULO VARCHAR(100),
    TIPO VARCHAR(100),
    ATAQUE int,
    DEFESA int,
    MAGIA int,
    DIFICULDADE int,
    HP float,
    HPPORNIVEL float,
    MP float,
    MPPORNIVEL float,
    VELOCIDADEMOVIMENTO float,
    ARMADURA float,
    ARMADURAPORNIVEL float,
    MR float,
    MRPORNIVEL float,
    RANGE float,
    REGENERACAOHP float,
    REGENERACAOHPPORNIVEL float,
    REGENERACAOMP float,
    REGENERACAOMPPORNIVEL float,
    CRITICO float,
    CRITICOPORNIVEL float,
    DANODEATAQUE float,
    DANODEATAQUEPORNIVEL float,
    VELOCIDADEDEATAQUE float,
    VELOCIDADEDEATAQUEPORNIVEL float,
    IMAGEM VARCHAR(200)
);
'''

# Executar o comando de criação de tabela
cursor.execute(create_table_query)

# Confirmar a transação
db_params.commit()

#Fechar conexão
if cursor:
    cursor.close()

In [7]:
#Inserindo dados na tabela

cursor = db_params.cursor()

delete_table_query = '''
Delete from tabelacampeao 
    '''

# Executando a consulta com valores reais
cursor.execute(delete_table_query)

db_params.commit()

#Fechar conexão
if cursor:
    cursor.close()

In [8]:
#Extraindo os dados para subir no banco

values = [
    (
        df_campeao.id_unico[i], df_campeao.versao_api[i], df_campeao.id[i], df_campeao.chave[i], df_campeao.nome[i], df_campeao.titulo[i], df_campeao.tipo[i], int(df_campeao.ataque[i]), int(df_campeao.defesa[i]), int(df_campeao.magia[i]), 
        int(df_campeao.dificuldade[i]), float(df_campeao.hp[i]), float(df_campeao.hp_por_nivel[i]), float(df_campeao.mp[i]), float(df_campeao.mp_por_nivel[i]), float(df_campeao.velocidade_movimento[i]), float(df_campeao.armadura[i]), 
        float(df_campeao.armadura_por_nivel[i]), float(df_campeao.mr[i]), float(df_campeao.mr_por_nivel[i]), float(df_campeao.ataque_range[i]), float(df_campeao.regeneracao_hp[i]), float(df_campeao.regeneracao_hp_nivel[i]), 
        float(df_campeao.regeneracao_mp[i]), float(df_campeao.renegeracao_mp_nivel[i]), float(df_campeao.critico[i]), float(df_campeao.critico_por_nivel[i]), float(df_campeao.dano_ataque[i]), float(df_campeao.dano_ataque_por_nivel[i]), 
        float(df_campeao.velocidade_ataque[i]), float(df_campeao.velocidade_ataque_por_nivel[i]),  df_campeao.imagem[i]
    )
    for i in range(len(df_campeao.id_unico))
]



In [9]:
#Inserindo dados na tabela

cursor = db_params.cursor()

insert_table_query = '''
INSERT INTO tabelacampeao (ID_UNICO, VERSAO_API, ID, CODIGO_CAMPEAO, NOME_CAMPEAO, TITULO, TIPO, ATAQUE, DEFESA, MAGIA, DIFICULDADE, HP, HPPORNIVEL, MP, MPPORNIVEL, VELOCIDADEMOVIMENTO,  ARMADURA, ARMADURAPORNIVEL, MR, MRPORNIVEL, RANGE, REGENERACAOHP, 
                            REGENERACAOHPPORNIVEL, REGENERACAOMP, REGENERACAOMPPORNIVEL, CRITICO, CRITICOPORNIVEL, DANODEATAQUE, DANODEATAQUEPORNIVEL, VELOCIDADEDEATAQUE, VELOCIDADEDEATAQUEPORNIVEL, IMAGEM)
   VALUES %s
    '''

# Executando a consulta com valores reais
execute_values(cursor, insert_table_query, values)

db_params.commit()

#Fechar conexão
if cursor:
    cursor.close()   